In [1]:
import numpy as np
import pandas as pd
from scipy import io
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

y_tr = pd.read_csv('datasets/tox21_labels_train.csv.gz',
                   index_col=0, compression="gzip")
y_te = pd.read_csv('datasets/tox21_labels_test.csv.gz', index_col=0, compression="gzip")
x_tr_dense = pd.read_csv('datasets/tox21_dense_train.csv.gz',
                         index_col=0, compression="gzip").values
x_te_dense = pd.read_csv('datasets/tox21_dense_test.csv.gz',
                         index_col=0, compression="gzip").values
x_tr_sparse = io.mmread('datasets/tox21_sparse_train.mtx.gz').tocsc()
x_te_sparse = io.mmread('datasets/tox21_sparse_test.mtx.gz').tocsc()

sparse_col_idx = ((x_tr_sparse > 0).mean(0) > 0.05).A.ravel()
x_tr = np.hstack([x_tr_dense, x_tr_sparse[:, sparse_col_idx].A])
x_te = np.hstack([x_te_dense, x_te_sparse[:, sparse_col_idx].A])


In [2]:
for target in y_tr.columns:
    rows_tr = np.isfinite(y_tr[target]).values
    rows_te = np.isfinite(y_te[target]).values
    rf = RandomForestClassifier(n_estimators=100,  n_jobs=4)
    rf.fit(x_tr[rows_tr], y_tr[target][rows_tr])
    p_te = rf.predict_proba(x_te[rows_te])
    auc_te = roc_auc_score(y_te[target][rows_te], p_te[:, 1])
    print("%15s: %3.5f" % (target, auc_te))


         NR.AhR: 0.90218
          NR.AR: 0.74303
      NR.AR.LBD: 0.72398
   NR.Aromatase: 0.79453
          NR.ER: 0.76740
      NR.ER.LBD: 0.77168
  NR.PPAR.gamma: 0.71847
         SR.ARE: 0.77049
       SR.ATAD5: 0.80648
         SR.HSE: 0.79669
         SR.MMP: 0.92329
         SR.p53: 0.78562
